In [1]:
"""********************************************************
 works fine
********************************************************"""
__author__ = 'Jingyi'

print(__doc__)

import distances_conversions
import Read_data
import class_spectrum
import numpy as np
import pylab
import pdb
import pyphot
from scipy import signal
import matplotlib.pyplot as plt

filters_directory='/home/jinlng/test_dir/Type_IIn/Filters/Filters'
color_dict={}
color_dict['NOT']='r'
color_dict['LT']='b'
color_dict['P60']='#e17701'
color_dict['P200']='#ff028d'
color_dict['LCOGT1M']='C'
color_dict['APO']='y'
color_dict['Keck1']='g'
color_dict['WHT']='M'

### ZTF18aavskep ###
z = 0.046   #CHECKED     
linesx = {r'$H_{\alpha}$': 6563, r'$H_{\beta}$': 4861, r'$H_{\gamma}$': 4341}
    #, r'$H_{\delta}$': 4102,r'$H_{\epsilon}$': 3970, r'$H_{\zeta}$': 3889, r'$H_{\eta}$': 3835}
distance_modulus = 36.5       #CHECKED
distance_pc = distances_conversions.DM_to_pc(distance_modulus)   # sn distance in pc
EBV = 0.035641      #CHECKED
explosion_date = 2458257.527317809 # CHECKED
data_dicts = Read_data.read_data_Marshall_simple(
    '/home/jinlng/test_dir/Type_IIn/ZTF18aavskep/data_Marshal.txt',
    filters_directory=filters_directory)

P48_r_array = np.zeros((np.shape(data_dicts['jd'][data_dicts['filter']=='r_p48'])[0],2))
P48_r_array[:,0] = data_dicts['jd'][data_dicts['filter']=='r_p48']
P48_r_array[:,1] = data_dicts['flux'][data_dicts['filter']=='r_p48']

### Calibration ###
### 1 ###
spec_1 = class_spectrum.spectrum(path_to_data=
    '/home/jinlng/test_dir/Type_IIn/ZTF18aavskep/ZTF18aavskep_20180602_P60_v1.ascii',
    instrument='P60',skiprows=0,time='2018-06-02T05:46:50.1',time_format='utc',show=False)
spec_1_cal = spec_1.calibrate(array_of_calibrating_data=P48_r_array,
    filter_family='ztf_p48',filter_name='r_p48',filters_directory=filters_directory)

### 2 ###
### time not clear
spec_2 = class_spectrum.spectrum(path_to_data=
    '/home/jinlng/test_dir/Type_IIn/ZTF18aavskep/ZTF18aavskep_20180610_LT_v1.ascii',
    instrument='LT',skiprows=0,time='2018-06-10T00:38:07.348',time_format='utc',show=False)
spec_2_cal = spec_2.calibrate(array_of_calibrating_data=P48_r_array,
    filter_family='ztf_p48',filter_name='r_p48',filters_directory=filters_directory)

### 3 ###
spec_3 = class_spectrum.spectrum(path_to_data=
    '/home/jinlng/test_dir/Type_IIn/ZTF18aavskep/ZTF18aavskep_20190119_NOT_v1.ascii',
    instrument='NOT',skiprows=0,time='2019-01-19T23:55:11.797',time_format='utc',show=False)    #time not clear
spec_3_cal = spec_3.calibrate(array_of_calibrating_data=P48_r_array,
    filter_family='ztf_p48',filter_name='r_p48',filters_directory=filters_directory)


spectra = [spec_1, spec_2, spec_3]
spectra_cal = [spec_1_cal, spec_2_cal, spec_3_cal]

dates = []
for i in spectra[::-1]:
    print('the date is',i.date_utc_hms())
    dates.append(i.date_utc_hms())

print('there are {0} spectra_cal in total'.format(len(spectra_cal)))

offsets = np.zeros(len(spectra))
for i in range(len(spectra_cal)):
    offsets[i] = i*np.mean(spectra_cal[i])

phases = ['+'+str(round(spectra[i].date_jd()-explosion_date,2)) for i in range(len(spectra))]
annotations = [(9300,-13.9),(8100,-14.5),(9700,-16.7)] #np.zeros[len(spectra_cal)]

### smoothing ###
signal_smooth = []
for i in range(len(spectra_cal)):
    signal_smooth.append(signal.savgol_filter(spectra_cal[i][:,1],53,7))

#plot all spectrum
fig = pylab.figure()
for i,speci in enumerate(spectra_cal):
    print('i is',i)
    print('offsets[i] is',offsets[i])
    print('spectra[i].date_utc_hms()',spectra[i].date_utc_hms())
    print('spectra[i].instrument',spectra[i].instrument)
    pylab.plot(speci[:, 0], np.log10(speci[:, 1]) + offsets[::-1][i],
               label=spectra[i].instrument + ', ' + spectra[i].date_utc_hms(), alpha=0.7)


import matplotlib.pyplot as plt
#plot all spectrum
fig = plt.figure()
for i,speci in enumerate(spectra_cal):
    print('i is',i)
    print('offsets[i] is',offsets[i])
    print('spectra[i].date_utc_hms()',spectra[i].date_utc_hms())
    print('spectra[i].instrument',spectra[i].instrument)
    plt.plot(speci[:, 0], speci[:, 1] + offsets[::-1][i],
               label=spectra[i].instrument + ', ' + spectra[i].date_utc_hms(), color=color_dict[spectra[i].instrument], alpha=0.7)
# plot each smoothed spectrum
for i,speci in enumerate(signal_smooth):
    plt.plot(spectra_cal[i][:, 0], speci + offsets[::-1][i], '-k', alpha=0.7)

for i,j in linesx.items():
    plt.axvline(j*(z+1),linestyle='-.',color='grey')
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
for i,j in enumerate(annotations):
    ax.annotate(phases[i],xy = (annotations[i]), fontsize=13)
plt.title('ZTF18aavskep', fontsize=20)
#plt.ylim(-17,-12.5)
plt.xlim(3000,11000)
plt.ylabel(r'$\rm{log(flux [erg/sec/cm^2/\AA ])}$', fontsize=20)
plt.xlabel(r'wavelength ($\AA$)', fontsize=20)
plt.grid()
plt.tight_layout()
plt.savefig('spectra_ZTF18aavskep.pdf', facecolor='w', edgecolor='w',orientation='portrait', papertype=None, format='pdf',transparent=False, bbox_inches='tight')
plt.show()


********************************************************
 works fine
********************************************************
path is /home/jinlng/test_dir/Type_IIn/ZTF18aavskep/data_Marshal.txt
row is ['2018 Mar 21', '2458198.9370', 'r', '99.0', '99.0', '99.0', '20.24', 'P48+ZTF', '1', 'None', 'None', 'True']
(12,)
2458198.9370
row is ['2018 Mar 21', '2458198.9791', 'r', '99.0', '99.0', '99.0', '20.31', 'P48+ZTF', '1', 'None', 'None', 'True']
(12,)
2458198.9791
row is ['2018 Apr 24', '2458232.8567', 'r', '99.0', '99.0', '99.0', '19.90', 'P48+ZTF', '1', 'None', 'None', 'True']
(12,)
2458232.8567
row is ['2018 May 05', '2458243.7957', 'r', '99.0', '99.0', '99.0', '19.91', 'P48+ZTF', '1', 'None', 'None', 'True']
(12,)
2458243.7957
row is ['2018 May 08', '2458246.7532', 'r', '99.0', '99.0', '99.0', '19.91', 'P48+ZTF', '1', 'None', 'None', 'True']
(12,)
2458246.7532
row is ['2018 May 17', '2458255.7931', 'r', '99.0', '99.0', '99.0', '20.09', 'P48+ZTF', '1', 'None', 'None', 'True']
(12,)
24

Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.63993220286

Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.639932202863843e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.63993220286

INFO:calibrate_spectrum_using_phot:output_path/txt_files did exist


Filter_object[0].AB_zero_flux is 4.74515228722624e-09 erg / angstrom * centimeter ** 2 * second
***
g_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'g_p48']]
Filter_object[0].AB_zero_flux is 4.74515228722624e-09 erg / angstrom * centimeter ** 2 * second
***
i_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'i_p48']]
Filter_object[0].AB_zero_flux is 1.7195294913482512e-09 erg / angstrom * centimeter ** 2 * second
***
i_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'i_p48']]
Filter_object[0].AB_zero_flux is 1.7195294913482512e-09 erg / angstrom * centimeter ** 2 * second
***
g_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'g_p48']]
Filter_object[0].AB_zero_flux is 4.74515228722624e-09 erg / angstrom * centimeter ** 2 * second
***
i_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'i_p48']]
Filter_object[0].AB_zero_flux is 1.7195294913482512e-09 erg / angstrom * centimeter ** 2 * second
***
r_p48
P48+ZTF
***
Filter_vector is [['ztf_p48', 'r_p48']]
Filter_object[0].AB_zero_flux is 2.63993220286

...

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [9]:
print (np.mean(spectra_cal[2][:,1]))

2.5163264665618327e-16
